<a href="https://colab.research.google.com/github/paulodowd/EMATM0053_21_22/blob/main/Labsheets/Core/L4_LineFollowing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: Line Following

In previous labsheets we have:

- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s).
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.

In this labsheet we will:
- Utilise prior work with the line sensors and motors to implement line following behaviour on the 3Pi+.
- Develop a **bang-bang** controller, using `logic` to control the robot.
- Implement a calibration routine for the ground sensor.  
- Develop a **weighted-measurement**, using a `proportional controller` to control the robot.


If you have prior experience with programming, you may wish to skip the section for the bang-bang controller.  The bang-bang controller is generally inefficient, but provides a good introduction to the concepts of providing feedback control to the 3Pi+.  It is still recommended to read through the bang-bang controller section regardless. 


<hr><br><br><br><br>

# Line Following, in Overview

In this labsheet, we will continue to work with just the 3 central ground sensors, labelled `DN2`, `DN3` and `DN4` on the 3PI+ and in the documentation.

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_BottomAnnotated.png?raw=true">
<br>
</p>

We can quickly sketch out a plan to produce line following behaviour from the knowledge we have so far.  In the illustration below, there are 3 examples of the robot in different scenarios with respect to the line:

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_LineFeedback.png?raw=true">
<br>
</p>

We can quickly draft the following logic:

- In **`scenario (A)`**, the central sensor is activated by the black line, and so the robot can continue with it's normal direction of travel.  This is the ideal scenario for line following.  Here, the ideal is to keep the black line between the inactivated outside sensors.  Note that, whilst we (humans) can observe the robot _will_ leave the line, presently the robot does not have this information.

- In **`scenario (B)`**, the robot has veered off the line to the **`left`**, causing the right-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn right`**.

- In **`scenario (C)`**, the robot has veered off the line to the **`right`**, causing the left-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn left`**.

In essence, we can write a controller to keep the black line between the left and right sensors.  Note, there are other ways to utilise this sensor to follow the line.  



## Exercise 1: Ground Sensors (15 minutes)

Feel free to discuss these questions with your peers:

1. Given your prior experience of the line-sensors, what do we expect the time-values of the 3 ground sensors to be on white and black surfaces?
  - in the above example, we have considered the central sensor to be "active" on the line, but we could also define this in inverted terms.  
  
2. What motion would you expect to observe in the robot motion if the feedback-signal was inverted?
  - is there a case where line following can be achieved with an inverted feedback signal?

3. An exceptional case not caught with the above logic would be if all three ground-sensors were detecting back simultaneously.  This can happen even though the black line is narrower than the gap between the two outer-most sensors.  Under what condition might this occur?

4. What would be appropriate feedback responses for other exceptional (non-defined) cases of the sensor activation?

5. If your robot was to calculate a performance score within `loop()` for line-following (a **`metric`**) as it operated:
  - what **`proprioceptive`** information could be used?
  - what **`exteroceptive`** information could be used?

<hr><br><br><br><br>

## Bang-Bang Controller

<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_PhotoShortLine.jpg?raw=true">
</p>

The most intuitive form of controller we can write for line following is a **`bang-bang`** controller.  A bang-bang controller has this name because the control decisions tend to be sudden and dramatic (bang!), moving between discrete states (bang<->bang).  

A consequence of this can be that there is no gradation in behaviour, or the behaviour looks jerky.  Usually, a bang-bang controller is not a good controller to use, but it is a good place to start if you have not programmed before.  It will help to develop a sense of how `conditional statements` can be used to branch code, to change the flow of your program, and so change the observable robot behaviour.  

A bang-bang controller is built using a series of **`if()`** statements.  We will use the `if()` statement to check if a condition is true, and if so, effect a specific behaviour.  

We can interpret the above section and write some **`pseudo-code`**:

```c
if( left_sensor == active_threshold ) {
  // Turn right.

} else if( right_sensor == active_threshold ) {
  // Turn left

} else if ( center_sensor == active_threshold ) {
  // Move forwards

} else {
  // Are there other conditions?
  // What is an appropriate response?

}
```

From Labsheet 3, you will have implemented a sensor reading function.  From Labsheet 2, you will have implemented some motor command functions.  



## Exercise 2: Implement a Bang-Bang Controller

1. Is a bang-bang controller **`open-loop`** or **`closed-loop`** control?  What is the difference?

2. Using your prior experience taking readings from the ground sensors, decide appropriate thresholds for when each sensor can be considered `active` or `inactive`:
  - decide whether large time measurements are active or inactive.
  - which sensor states will be used to build the bang-bang control logic discussed above?
  - are these threshold values the same for `DN2`, `DN3` and `DN4`?  Remember that you can inspect the values of your sensors using `Serial.print()` and `Serial.println()`.
  - it is recommended to use `#define` statements to set these values at the top of your program.

3. Implement the discussed bang-bang controller logic within your `loop()`:
  - use the black vinyl tape to create a short black line on your work surface.
  - it is recommended you start with `slow` or `low` motor velocities.
  - to begin with, do not implement forward motion.  Instead, work only with turning on the spot (rotation).
  - check that your `feedback signal` turns your robot in the appropriate directions with respect to sensor activation.
  - investigate what happens when your line is placed into difficult places with respect to the line - are any of these a problem you need to address?
  - once you are confident rotation is working properly, implement forward motion.
  - **help**: remember your conditional statement can use: 
    - `<`  less than
    - `<=` less than or equal to
    - `==` equal to
    - `>=` greater than or equal to
    - `>`  greater than
    - `!=` not equal to
  - **help**: what is the functional difference between the two code examples immediately below?
  
```c
  // Example 1
  if( ) {

  } 
  if( ) {

  }

  // Example 2
  if( ) {

  } else if( ) {

  } 
```

4. Does your robot conduct `turn` and `move fowards` operations seperately?  
  - Can these be integrated so that the robot does not stop moving forwards?
    - it may be useful to use your bang-bang logic to set the value of left and right `pwm` or `power` variables, which are then used to command the motors once after the logic.
  - How is performance effected with turning and moving forwards combined?
  - Moving slowly might increase the general reliability of the line following behaviour.  As a thought experiment, what other hypothetical **`task requirements`** would make fast forward speed desirable for the robotic system?
  - What is the quickest forward speed you can utilise and still achieve reliable line-following?
  - If you have not done so already, experiment with more challenging line shapes, such as corners and curves.  

5. What information about the line does the robot have when no sensors detect the black surface?
  - When might this circumstance occur?  Consider the line map provided for the line following challenge.
  - What would be an appropriate response in this condition?
  - What other information is available to the robot that might be useful?

6. Write a function to simply confirm if the robot is on a black line.  The function should report a `true` or `false` value when called.
  - is there a reason to discriminate between which of the sensors is on a black surface, or can it be any of the 3?  Explain your reasoning, adjust the function if necessary.

7. Use the above function to allow your robot to start off the line and drive forwards until it meets the line.  Once it is on the line, it should activate the bang-bang controller.
  - Consider using a `global` variable and an `if()` statement to switch the robot between these two behaviours. 

8. Start your robot off the line, and allow it to travel forward to join and follow the line.  Currently, what is the most extreme <a href="https://en.wikipedia.org/wiki/Angle_of_incidence_(optics)">angle of incidence</a> where your controller can still successfully begin line following?
  - if you were to create a results table of different angles when joining the line, how could you quantify the reliability of the controller?

